In [3]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import lightgbm as lgb
from scipy.stats import spearmanr
from tqdm import tqdm
from sklearn.utils import resample

In [4]:
df = pd.read_parquet('protein_embeddings_q3.parquet')

# Adding supplementary features to the dataset

In [5]:
aa_properties = {
    # AA: {prop1: val1, prop2: val2, ...}
    'A': {'hydro_KD': 1.8,  'volume': 88.6,  'charge': 0, 'polarity': 8.1, 'flexibility': 0.357},
    'R': {'hydro_KD': -4.5, 'volume': 173.4, 'charge': 1, 'polarity': 10.5, 'flexibility': 0.529},
    'N': {'hydro_KD': -3.5, 'volume': 114.1, 'charge': 0, 'polarity': 11.6, 'flexibility': 0.463},
    'D': {'hydro_KD': -3.5, 'volume': 111.1, 'charge': -1, 'polarity': 13.0, 'flexibility': 0.511},
    'C': {'hydro_KD': 2.5,  'volume': 108.5, 'charge': 0, 'polarity': 5.5, 'flexibility': 0.346},
    'Q': {'hydro_KD': -3.5, 'volume': 143.8, 'charge': 0, 'polarity': 10.5, 'flexibility': 0.493},
    'E': {'hydro_KD': -3.5, 'volume': 138.4, 'charge': -1, 'polarity': 12.3, 'flexibility': 0.497},
    'G': {'hydro_KD': -0.4, 'volume': 60.1,  'charge': 0, 'polarity': 9.0, 'flexibility': 0.544},
    'H': {'hydro_KD': -3.2, 'volume': 153.2, 'charge': 0, 'polarity': 10.4, 'flexibility': 0.323}, # Note: Charge is pH dependent; 0 here is simplified neutral form
    'I': {'hydro_KD': 4.5,  'volume': 166.7, 'charge': 0, 'polarity': 5.2, 'flexibility': 0.462},
    'L': {'hydro_KD': 3.8,  'volume': 166.7, 'charge': 0, 'polarity': 4.9, 'flexibility': 0.365},
    'K': {'hydro_KD': -3.9, 'volume': 168.6, 'charge': 1, 'polarity': 11.3, 'flexibility': 0.466},
    'M': {'hydro_KD': 1.9,  'volume': 162.9, 'charge': 0, 'polarity': 5.7, 'flexibility': 0.295},
    'F': {'hydro_KD': 2.8,  'volume': 189.9, 'charge': 0, 'polarity': 5.2, 'flexibility': 0.314},
    'P': {'hydro_KD': -1.6, 'volume': 112.7, 'charge': 0, 'polarity': 8.0, 'flexibility': 0.509}, # Proline is conformationally restricted
    'S': {'hydro_KD': -0.8, 'volume': 89.0,  'charge': 0, 'polarity': 9.2, 'flexibility': 0.507},
    'T': {'hydro_KD': -0.7, 'volume': 116.1, 'charge': 0, 'polarity': 8.6, 'flexibility': 0.444},
    'W': {'hydro_KD': -0.9, 'volume': 227.8, 'charge': 0, 'polarity': 5.4, 'flexibility': 0.305},
    'Y': {'hydro_KD': -1.3, 'volume': 193.6, 'charge': 0, 'polarity': 6.2, 'flexibility': 0.420},
    'V': {'hydro_KD': 4.2,  'volume': 140.0, 'charge': 0, 'polarity': 5.9, 'flexibility': 0.386},
    # Handle non-standard/unknown amino acids if necessary
    'X': {'hydro_KD': np.nan, 'volume': np.nan, 'charge': np.nan, 'polarity': np.nan, 'flexibility': np.nan},
    '-': {'hydro_KD': np.nan, 'volume': np.nan, 'charge': np.nan, 'polarity': np.nan, 'flexibility': np.nan} # Gap character
}


def calculate_features_with_props(row, properties_dict):
    wt_aa = row['WT_AA']
    mut_aa = row['Mutant_AA']
    features = {}

    # Get properties
    wt_props = properties_dict.get(wt_aa, properties_dict.get('X')) # Default to 'X' if AA not found
    mut_props = properties_dict.get(mut_aa, properties_dict.get('X'))

    # Create features for each property
    for prop_name in wt_props.keys(): # Iterate through property names (hydro_KD, volume, etc.)
        wt_val = wt_props[prop_name]
        mut_val = mut_props.get(prop_name, np.nan) # Get corresponding mutant prop

        features[f'wt_{prop_name}'] = wt_val
        features[f'mut_{prop_name}'] = mut_val

        # Calculate the difference (Delta feature)
        if pd.notna(wt_val) and pd.notna(mut_val):
            features[f'delta_{prop_name}'] = mut_val - wt_val
        else:
            features[f'delta_{prop_name}'] = np.nan

    # ... add other features like position, one-hot encoding, etc. ...
    features['position'] = row['Position']

    return pd.Series(features)


def add_physical(df):
    df['WT_AA'] = df["mutant"].apply(lambda seq: seq[0])
    df['Mutant_AA'] = df["mutant"].apply(lambda seq: seq[-1])
    df['Position'] = df["mutant"].apply(lambda seq: int(seq[1:-1]))

    feature_df = df.apply(lambda row: calculate_features_with_props(row, aa_properties), axis=1)
    return feature_df

In [6]:
feature_df = add_physical(df)
df = pd.concat((df, feature_df),axis=1)

In [7]:
import numpy as np

def parse_pssm(pssm_path):
    pssm = []
    with open(pssm_path, "r") as f:
        start = False
        for line in f:
            if line.strip().startswith("Last position-specific scoring matrix computed"):
                start = True
                next(f)  # skip header
                continue
            if start:
                if not line.strip():
                    break
                values = line.strip().split()
                scores = [int(x) for x in values[2:22]]  # 20 AA scores
                pssm.append(scores)
    return np.array(pssm)  # shape: [L, 20]


aa_to_index = {
    'A': 0, 'R': 1, 'N': 2, 'D': 3, 'C': 4,
    'Q': 5, 'E': 6, 'G': 7, 'H': 8, 'I': 9,
    'L': 10, 'K': 11, 'M': 12, 'F': 13, 'P': 14,
    'S': 15, 'T': 16, 'W': 17, 'Y': 18, 'V': 19
}

def get_pssm_features(pssm, pos, wt_aa, mut_aa):
    wt_idx = aa_to_index[wt_aa]
    mut_idx = aa_to_index[mut_aa]
    wt_score = pssm[pos][wt_idx]
    mut_score = pssm[pos][mut_idx]
    delta = mut_score - wt_score
    return [wt_score, mut_score, delta]


def get_pssm_df(df):
    pssm = parse_pssm('pssm.txt')
    result = df.apply(
        lambda row: pd.Series(
            get_pssm_features(
                pssm=pssm,
                pos=row['Position'],
                wt_aa=row['mutant'][0],    # First character is wild-type AA
                mut_aa=row['mutant'][-1]   # Last character is mutant AA
            ),
            index=['wt_score', 'mut_score', 'delta_score']
        ),
        axis=1
    )
    
    return result

pssm_df = get_pssm_df(df)

In [8]:
df = pd.concat((df, pssm_df), axis=1)

In [9]:

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X = np.vstack(df["Embedding"].values)
X = np.concatenate((X,feature_df.values), axis=1)
X = np.concatenate((X,pssm_df.values), axis=1)

y = df["DMS_score"].values

# Convert to PyTorch tensors and move to GPU
X_tensor = torch.tensor(X, dtype=torch.float32, device=device)
y_tensor = torch.tensor(y, dtype=torch.float32, device=device).view(-1, 1)  # Reshape for MLP

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Training Ensemble of LightGBM + Ridge Regression

In [10]:

# Define hyperparameters for LightGBM
lgb_params = {"objective":"regression", "metric": "rmse", 'learning_rate': 0.0436032560500989,  'num_leaves': 76, 'min_child_samples': 6, 'max_depth': 7, 'subsample': 0.5939560221912161, 'colsample_bytree': 0.6618044618804528, }



# Number of bootstrapped models
num_bootstraps = 10
lgb_boot_models = []
lgb_predictions = []

progress = tqdm(total=num_bootstraps, desc="Training Bootstrapped LightGBM Models")

for i in range(num_bootstraps):
    # Bootstrap resampling
    X_resampled, y_resampled = resample(X,y, replace=True, n_samples=X_train.shape[0], random_state=i)

    # Train LightGBM model on resampled data
    lgb_train = lgb.Dataset(X_resampled, label=y_resampled)
    lgb_model = lgb.train(lgb_params, lgb_train, num_boost_round=100)
    
    # Store trained model
    lgb_boot_models.append(lgb_model)

    # Predict on test set
    y_pred_lgb = lgb_model.predict(X_test.cpu().numpy())
    lgb_predictions.append(y_pred_lgb)

    progress.update(1)

progress.close()

# Average predictions from bootstrapped models
y_pred_ensemble = np.mean(np.column_stack(lgb_predictions), axis=1)

# Compute Spearman correlation
spearman_corr_ensemble, _ = spearmanr(y_test.cpu().numpy().flatten(), y_pred_ensemble)

print(f"Bootstrapped LightGBM Ensemble Spearman Correlation: {spearman_corr_ensemble:.4f}")


Training Bootstrapped LightGBM Models:   0%|          | 0/10 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016602 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 326025
[LightGBM] [Info] Number of data points in the train set: 1152, number of used features: 1299
[LightGBM] [Info] Start training from score 0.349639
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

Training Bootstrapped LightGBM Models:  10%|█         | 1/10 [00:03<00:35,  3.95s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015353 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 326085
[LightGBM] [Info] Number of data points in the train set: 1152, number of used features: 1299
[LightGBM] [Info] Start training from score 0.336406
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

Training Bootstrapped LightGBM Models:  20%|██        | 2/10 [00:07<00:30,  3.77s/it]

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014795 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 326100
[LightGBM] [Info] Number of data points in the train set: 1152, number of used features: 1299
[LightGBM] [Info] Start training from score 0.329540
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

Training Bootstrapped LightGBM Models:  90%|█████████ | 9/10 [00:31<00:03,  3.45s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015850 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 325980
[LightGBM] [Info] Number of data points in the train set: 1152, number of used features: 1299
[LightGBM] [Info] Start training from score 0.344190
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

Training Bootstrapped LightGBM Models: 100%|██████████| 10/10 [00:40<00:00,  4.01s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Bootstrapped LightGBM Ensemble Spearman Correlation: 0.9266


In [11]:
# Step 2: Train Meta Predictor

from sklearn.linear_model import Ridge

progress = tqdm(total=len(lgb_boot_models), desc="Generating `X_meta_train`")
X_meta_train = []
for m in lgb_boot_models:
    X_meta_train.append(m.predict(X_train.cpu().numpy()))
    progress.update(1)

progress.close()

X_meta_train = np.column_stack(X_meta_train)
y_meta_train = y_train.cpu().numpy().flatten()


# Train a Ridge Regression as the meta-model
meta_model = Ridge(alpha=1.0)
meta_model.fit(X_meta_train, y_meta_train)

## EVAL::
# Predict with stacked model
progress = tqdm(total=len(lgb_boot_models), desc="Predicting for `X_meta_test`")
X_meta_test = []
for m in lgb_boot_models:
    X_meta_test.append(m.predict(X_test.cpu().numpy()))
    progress.update(1)
progress.close()
X_meta_test = np.column_stack(X_meta_test)
y_meta_test = y_train.cpu().numpy().flatten()

y_meta_pred = meta_model.predict(X_meta_train)

# Compute Spearman correlation
spearman_corr_stacked, _ = spearmanr(y_meta_test, y_meta_pred)
print(f"Meta LightGBM Model Spearman Correlation: {spearman_corr_stacked:.4f}")


Predicting for `X_meta_test`: 100%|██████████| 10/10 [00:00<00:00, 668.34it/s]

Meta LightGBM Model Spearman Correlation: 0.9346


In [12]:
df_test = pd.read_parquet('protein_embeddings_test.parquet')
X_unlabeled = df_test['Embedding'].values
X_unlabeled = np.vstack(X_unlabeled)
feature_df_test = add_physical(df_test)
pssm_df_test = get_pssm_df(df_test)
X_unlabeled = np.concatenate((X_unlabeled, feature_df_test),axis=1)
X_unlabeled = np.concatenate((X_unlabeled, pssm_df_test), axis=1)

X_unlabeled = torch.tensor(X_unlabeled, dtype=torch.float32, device=device)


In [13]:

# Compute UCB for LightGBM ensemble

y_pred_all = np.column_stack([model.predict(X_unlabeled.cpu().numpy()) for model in lgb_boot_models])
std_pred = np.std(y_pred_all, axis=1)    # Uncertainty (std dev)
mean_pred = meta_model.predict(y_pred_all)

In [14]:
df_test["DMS_score_predicted"] = mean_pred
df_test[["mutant", "DMS_score_predicted"]].to_csv("predictions.csv", index=False)